In [ ]:
import pandas as pd
import numpy as np
import tensorflow as tf
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from tensorflow.lite.python.util import convert_bytes_to_c_source
import os

# 1. LOAD DATA SAFELY
filename = 'indian_liver_patient.csv'
if not os.path.exists(filename):
    print(f"❌ ERROR: {filename} not found!")
    print("Please drag and drop the CSV file into the folder icon on the left.")
else:
    print(f"✅ Found {filename}!")
    df = pd.read_csv(filename)

    # 2. CLEAN DATA
    # Rename columns to be easier to work with
    df.columns = ['Age', 'Gender', 'TB', 'DB', 'Alkphos', 'Sgpt', 'Sgot', 'TP', 'Albumin', 'AG', 'Selector']

    # Convert Gender to numbers (Female=0, Male=1)
    df['Gender'] = df['Gender'].apply(lambda x: 0 if x == 'Female' else 1)

    # Fill missing values with 0
    df = df.fillna(0)

    # Define Inputs (X) and Output (y)
    X = df.drop('Selector', axis=1)
    y = df['Selector']

    # Remap Output: Original dataset has 1=Disease, 2=Healthy.
    # We change it to: 1=Disease, 0=Healthy for better AI training
    y = y.replace(2, 0)

    # 3. GET SCALER VALUES (Save these for your Arduino code!)
    scaler = StandardScaler()
    X_scaled = scaler.fit_transform(X)

    print("\n--- COPY THESE NUMBERS FOR ARDUINO ---")
    print("float MEANS[] = {", ", ".join(map(str, scaler.mean_)), "};")
    print("float SCALES[] = {", ", ".join(map(str, scaler.scale_)), "};")
    print("----------------------------------------\n")

    # 4. TRAIN MODEL
    X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2, random_state=42)

    model = tf.keras.Sequential([
        tf.keras.layers.Dense(10, activation='relu', input_shape=(10,)), # Input layer
        tf.keras.layers.Dense(10, activation='relu'),                    # Hidden layer
        tf.keras.layers.Dense(1, activation='sigmoid')                   # Output layer
    ])

    model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
    model.fit(X_train, y_train, epochs=100, batch_size=10, verbose=0)
    print("✅ Model Trained!")

    # 5. EXPORT FOR ARDUINO (The Critical Step)
    converter = tf.lite.TFLiteConverter.from_keras_model(model)
    # CRITICAL: We do NOT use optimizations here to prevent NaN errors
    converter.optimizations = []
    tflite_model = converter.convert()

    # Convert to C header format
    source_text, header_text = convert_bytes_to_c_source(
        tflite_model,
        "model_data",
        include_guard="MODEL_DATA_H"
    )

    print("\n✅ GENERATING ARDUINO CODE...")
    print("👇👇👇 COPY THE ARRAY BELOW INTO model.h 👇👇👇\n")
    print(header_text)

✅ Found indian_liver_patient.csv!

--- COPY THESE NUMBERS FOR ARDUINO ---
float MEANS[] = { 44.74614065180103, 0.7564322469982847, 3.298799313893653, 1.486106346483705, 290.57632933104634, 80.71355060034305, 109.91080617495712, 6.483190394511149, 3.141852487135506, 0.9405660377358489 };
float SCALES[] = { 16.175942411270466, 0.429234787382629, 6.204193950230359, 2.806087923849646, 242.72954813780882, 182.46366758318712, 288.67063666027326, 1.0845201655473806, 0.7948362500202801, 0.3276809549641565 };
----------------------------------------



/usr/local/lib/python3.12/dist-packages/keras/src/layers/core/dense.py:93: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


✅ Model Trained!
Saved artifact at '/tmp/tmp90q6107c'. The following endpoints are available:

* Endpoint 'serve'
  args_0 (POSITIONAL_ONLY): TensorSpec(shape=(None, 10), dtype=tf.float32, name='keras_tensor')
Output Type:
  TensorSpec(shape=(None, 1), dtype=tf.float32, name=None)
Captures:
  135875281338000: TensorSpec(shape=(), dtype=tf.resource, name=None)
  135875281338192: TensorSpec(shape=(), dtype=tf.resource, name=None)
  135875281336272: TensorSpec(shape=(), dtype=tf.resource, name=None)
  135877457924496: TensorSpec(shape=(), dtype=tf.resource, name=None)
  135875281337040: TensorSpec(shape=(), dtype=tf.resource, name=None)
  135877872732880: TensorSpec(shape=(), dtype=tf.resource, name=None)

✅ GENERATING ARDUINO CODE...
👇👇👇 COPY THE ARRAY BELOW INTO model.h 👇👇👇




// This is a TensorFlow Lite model file that has been converted into a C data
// array using the tensorflow.lite.util.convert_bytes_to_c_source() function.
// This form is useful for compiling into a binary for d

In [ ]:
# --- RUN THIS TO SEE THE MISSING DATA ---
print(source_text)


// This is a TensorFlow Lite model file that has been converted into a C data
// array using the tensorflow.lite.util.convert_bytes_to_c_source() function.
// This form is useful for compiling into a binary for devices that don't have a
// file system.


// We need to keep the data array aligned on some architectures.
#ifdef __has_attribute
#define HAVE_ATTRIBUTE(x) __has_attribute(x)
#else
#define HAVE_ATTRIBUTE(x) 0
#endif
#if HAVE_ATTRIBUTE(aligned) || (defined(__GNUC__) && !defined(__clang__))
#define DATA_ALIGN_ATTRIBUTE __attribute__((aligned(4)))
#else
#define DATA_ALIGN_ATTRIBUTE
#endif

const unsigned char model_data[] DATA_ALIGN_ATTRIBUTE = {
    0x1c, 0x00, 0x00, 0x00, 0x54, 0x46, 0x4c, 0x33, 0x14, 0x00, 0x20, 0x00, 0x1c,
    0x00, 0x18, 0x00, 0x14, 0x00, 0x10, 0x00, 0x0c, 0x00, 0x00, 0x00, 0x08, 0x00,
    0x04, 0x00, 0x14, 0x00, 0x00, 0x00, 0x1c, 0x00, 0x00, 0x00, 0x8c, 0x00, 0x00,
    0x00, 0xe4, 0x00, 0x00, 0x00, 0xa8, 0x05, 0x00, 0x00, 0xb8, 0x05, 0x00, 0x00,
    0x4c, 